In [1]:
def prepare_index(filename, index_path, node2id_path, id2node_path, emb_col=1, delimiter=' ', dim=100,
                  distance='euclidean', trees=10, redo=False):
    if redo:
        t = AnnoyIndex(dim, distance)  # Length of item vector that will be indexed
        node2id={}
        id2node={}
        with gzip.open(filename, 'r') as f:
            i=0
            header=next(f)
            for line in f:
                line=line.decode()
                line_data=line.split()
                if line_data[1]=='embedding_sentence': continue
                node=line_data[0]
                if emb_col==2:
                    data=line_data[emb_col].split(delimiter)
                else:
                    data=line_data[emb_col:]
                v=[float(d) for d in data]
                
                t.add_item(i, v)
                node2id[node]=i
                id2node[i]=node
                i+=1

        t.build(trees) # number of trees (more -> higher precision at query time)
        t.save(graph_emb_index)
        
        pkl.dump(node2id, open(node2id_path, 'wb'))
        pkl.dump(id2node, open(id2node_path, 'wb'))
    else:
        node2id=pkl.load(open(node2id_path, 'rb'))
        id2node=pkl.load(open(id2node_path, 'rb'))
    
        t = AnnoyIndex(dim, distance)
        t.load(graph_emb_index) # super fast, will just mmap the file

    
    return t, node2id, id2node

In [ ]:
graph_index, graph_node2id, graph_id2node = prepare_index(graph_emb_path,
                                              graph_index_path,
                                              graph_node2id_path,
                                              graph_id2node_path,
                                              graph_emb_col,
                                              graph_emb_del,
                                              graph_dim,
                                              distance,
                                              graph_trees,
                                              redo
                                             )

In [ ]:
text_index, text_node2id, text_id2node = prepare_index(text_emb_path,
                                              text_index_path,
                                              text_node2id_path,
                                              text_id2node_path,
                                              text_emb_col,
                                              text_emb_del,
                                              text_dim,
                                              distance,
                                              text_trees,
                                              redo
                                             )

In [ ]:
def obtain_similar_nodes(node, index, node2id, id2node, num_nodes=10):
    node_id=node2id[node]
    return [id2node[i] for i in index.get_nns_by_item(node_id, num_nodes+1)[1:]]

In [ ]:
nodes=['/c/en/turtle', '/c/en/happy', '/c/en/turtle/n/wn/animal']

In [ ]:
for node in nodes:
    
    print('GRAPH EMBEDDING DATA\n')
    graph_sim=obtain_similar_nodes(node, graph_index, graph_node2id, graph_id2node)
    print(node, graph_sim, '\n')
    
    print('TEXT EMBEDDING DATA\n')